In [7]:
import os
import json
import pandas as pd
import numpy as np
import psycopg2
import joblib
import logging
from typing import List
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.metrics import classification_report
from sklearn.neighbors import LocalOutlierFactor
import logging
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import (
    r2_score,
    explained_variance_score,
    mean_absolute_error,
    mean_squared_error
)
import torch
import torch.nn as nn
from typing import Optional
# Set options to show full DataFrame output
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

conn = psycopg2.connect(host='fortifai-ng-dev-db.postgres.database.azure.com',
			database='baldota-dev-db',
			user='fortifai_ng_ai_user_rw',
			password='AIPwd@123!',
			port='5432',
            sslmode="require"
		)
cur = conn.cursor()
cur.execute("""
            SELECT table_name 
            FROM information_schema.tables 
        """)

tables = [row[0] for row in cur.fetchall()]
po_data= pd.read_sql_query(f"SELECT * FROM staging_db.{'po_header_lineitem_merged_with_risks'}", conn)
cur.close()
conn.close()
#p2p_header_po_data.head()
print(po_data.shape)
po_data.head()

C:\Users\ajayn\AppData\Local\Temp\ipykernel_29156\2997526742.py:43: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  po_data= pd.read_sql_query(f"SELECT * FROM staging_db.{'po_header_lineitem_merged_with_risks'}", conn)


(15007, 71)


,purch_doc_no_src_po,purch_doc_item_no_src_po,pr_no_src_po,pr_item_no_src_po,requester_name_src_po,doc_change_date_src_po,company_code_src_po,p2o_unit_conv_denom_src_po,o2b_unit_conv_denom_src_po,p2o_unit_conv_num_src_po,o2b_unit_conv_num_src_po,po_item_del_flag_src_po,gr_indicator_src_po,gr_invoice_verif_flag_src_po,gross_val_po_curr_src_po,inv_receipt_indicator_src_po,material_type_src_po,material_no_src_po,matl_group_src_po,net_price_doc_curr_src_po,net_val_po_curr_src_po,order_uom_src_po,quantity_src_po,plant_src_po,short_text_src_po,target_qty_src_po,outline_agrmt_tgt_val_doc_curr_src_po,reqmt_tracking_no_src_po,principal_purch_agrmt_item_no_src_po,base_id_src_po,purch_doc_no_hpd_po,vendor_or_creditor_acct_no_hpd_po,purch_doc_category_hpd_po,company_code_hpd_po,currency_hpd_po,object_created_by_hpd_po,doc_change_date_hpd_po,control_indicator_hpd_po,purch_doc_date_hpd_po,principal_purch_agrmt_no_hpd_po,exchange_rate_hpd_po,purch_group_hpd_po,purch_org_hpd_po,pymnt_terms_hpd_po,processing_status_hpd_po,doc_release_incompl_flag_hpd_po,release_indicator_hpd_po,release_status_hpd_po,rfq_status_hpd_po,resp_vendor_salesperson_hpd_po,on_release_total_value_hpd_po,purch_doc_type_hpd_po,gbt_fraud_score,gbt_model_flag,ae_fraud_score,ae_predicted_flag,iso_fraud_score,iso_predicted_flag,risk_score,risk_level,model_flag,predicted_risks,main_risk_scenario,sub_risk_1,sub_risk_2,impact,impact_1,impact_2,impact_3,risk_summary_object,vendor_name_1
0,4000003550,00010,12009695,00010,BADRINATHA,2024-05-03,ABFS,0.0,0.0,1.0,1.0,None,X,X,300000.00,X,None,None,6100,300000.00,300000.00,AU,1.0,CT01,Special Design Consultancy for Textile P,0.0,0.0,01,0.0,4000003550^00010,4000003550,2301956,F,ABFS,INR,BADRMMCORP,2024-05-03,None,2025-04-03,1000270362,1.0,CSR,CHTR,1,5,None,R,XX,9,MK,300000.00,ZREG,0.000727,0,0.029215,0,-0.302036,0,0.001021,No Risk,0,(No Risk),No Risk,No Risk,No Risk,[None],No Risk,No Risk,No Risk,There is No Risk for this line item.,MAYANK KAUL
1,4000003551,00010,12009706,00010,BADRINATHA,2024-05-03,MSPL,0.0,0.0,1.0,1.0,None,X,X,29734.78,X,None,None,6220,29734.78,29734.78,AU,1.0,AV01,"LANDING, PARKING, GA & FMC",0.0,0.0,001,0.0,4000003551^00010,4000003551,2300636,F,MSPL,INR,BADRMMCORP,2024-05-03,None,2025-04-03,1000270560,1.0,AVI,AVIA,1,5,None,R,XX,9,MIAL,90326.12,ZREG,0.001051,0,0.029969,0,-0.301779,0,0.000786,No Risk,0,(No Risk),No Risk,No Risk,No Risk,[None],No Risk,No Risk,No Risk,There is No Risk for this line item.,MUMBAI INTERNATIONAL AIRPORT LTD
2,4000003551,00020,12009706,00020,BADRINATHA,2024-05-03,MSPL,0.0,0.0,1.0,1.0,None,X,X,300.00,X,None,None,6220,300.00,300.00,AU,1.0,AV01,DEVELOPMENT FEE,0.0,0.0,002,0.0,4000003551^00020,4000003551,2300636,F,MSPL,INR,BADRMMCORP,2024-05-03,None,2025-04-03,1000270560,1.0,AVI,AVIA,1,5,None,R,XX,9,MIAL,90326.12,ZREG,0.001051,0,0.030980,0,-0.306114,0,0.000736,No Risk,0,(No Risk),No Risk,No Risk,No Risk,[None],No Risk,No Risk,No Risk,There is No Risk for this line item.,MUMBAI INTERNATIONAL AIRPORT LTD
3,4000003551,00030,12009694,00020,BADRINATHA,2024-05-03,MSPL,0.0,0.0,1.0,1.0,None,X,X,31455.70,X,None,None,6220,31455.70,31455.70,AU,1.0,AV01,"LANDING,PARKING, GA & FMC",0.0,0.0,01,0.0,4000003551^00030,4000003551,2300636,F,MSPL,INR,BADRMMCORP,2024-05-03,None,2025-04-03,1000270560,1.0,AVI,AVIA,1,5,None,R,XX,9,MIAL,90326.12,ZREG,0.001051,0,0.030487,0,-0.302538,0,0.000789,No Risk,0,(No Risk),No Risk,No Risk,No Risk,[None],No Risk,No Risk,No Risk,There is No Risk for this line item.,MUMBAI INTERNATIONAL AIRPORT LTD
4,4000003551,00040,12009694,00030,BADRINATHA,2024-05-03,MSPL,0.0,0.0,1.0,1.0,None,X,X,400.00,X,None,None,6220,400.00,400.00,AU,1.0,AV01,DEVELOPMENT FEE,0.0,0.0,02,0.0,4000003551^00040,4000003551,2300636,F,MSPL,INR,BADRMMCORP,2024-05-03,None,2025-04-03,1000270560,1.0,AVI,AVIA,1,5,None,R,XX,9,MIAL,90326.12,ZREG,0.001051,0,0.031166,0,-0.304998,0,0.000736,No Risk,0,(No Risk),No Risk,No Risk,No Risk,[None],No Risk,No Risk,No Risk,There is No Risk for this line item.,MUMBAI INTERNATIONAL AIRPORT 

In [9]:
vendor_or_creditor_acct_no_hpd_po - supplier
vendor_name_1 - Supplier name

material_no_src_po -  Material
short_text_src_po - Short Text

net_val_po_curr_src_po	- net_value
currency_hpd_po - currency

exchange_rate_hpd_po

NameError: name 'vendor_or_creditor_acct_no_hpd_po' is not defined

In [3]:
po_data.columns

Index(['purch_doc_no_src_po', 'purch_doc_item_no_src_po', 'pr_no_src_po',
       'pr_item_no_src_po', 'requester_name_src_po', 'doc_change_date_src_po',
       'company_code_src_po', 'p2o_unit_conv_denom_src_po',
       'o2b_unit_conv_denom_src_po', 'p2o_unit_conv_num_src_po',
       'o2b_unit_conv_num_src_po', 'po_item_del_flag_src_po',
       'gr_indicator_src_po', 'gr_invoice_verif_flag_src_po',
       'gross_val_po_curr_src_po', 'inv_receipt_indicator_src_po',
       'material_type_src_po', 'material_no_src_po', 'matl_group_src_po',
       'net_price_doc_curr_src_po', 'net_val_po_curr_src_po',
       'order_uom_src_po', 'quantity_src_po', 'plant_src_po',
       'short_text_src_po', 'target_qty_src_po',
       'outline_agrmt_tgt_val_doc_curr_src_po', 'reqmt_tracking_no_src_po',
       'principal_purch_agrmt_item_no_src_po', 'base_id_src_po',
       'purch_doc_no_hpd_po', 'vendor_or_creditor_acct_no_hpd_po',
       'purch_doc_category_hpd_po', 'company_code_hpd_po', 'currency_hpd_po'

In [6]:
# ============================================================
# FortifAI — Batch Reports (Daily / Weekly / Monthly) for 6 Months
# Output:
#   generate_reports/
#       Daily/<YYYY-MM-DD>/
#       Weekly/<YYYY>-W<WW>_<YYYY-MM-DD>_to_<YYYY-MM-DD>/
#       Monthly/<YYYY-MM>/
#
# Expects a DataFrame in memory: `po_data`
# Uses your report logic & column mappings verbatim.
# ============================================================
import os, re
import pandas as pd
from datetime import date, timedelta
from dateutil.relativedelta import relativedelta

# ------------------------------
# CONFIG
# ------------------------------
BASE_OUT     = "generate_reports"
TZ_NAME      = "Asia/Kolkata"
RUN_DATE     = pd.Timestamp.now(tz=TZ_NAME).date()
DATE_COL     = None  # set to your exact date column if known (post-rename not required; we detect on raw `po_data`)

# Toggle specific report families
RUN_TOP10_VENDORS_AND_TXNS = True
RUN_HIGHRISK_TXNS          = True
RUN_SPLIT_PO               = True
RUN_PRICE_VARIANCE         = True

# ------------------------------
# Excel engine & formatting utils
# ------------------------------
def pick_engine():
    try:
        import xlsxwriter  # noqa: F401
        return "xlsxwriter"
    except Exception:
        return "openpyxl"

ENGINE = pick_engine()

def writer_with_formats(path):
    w = pd.ExcelWriter(path, engine=ENGINE)
    if ENGINE == "xlsxwriter":
        wb = w.book
        fmt = {
            "money": wb.add_format({"num_format": "#,##0.00"}),
            "text":  wb.add_format({"num_format": "@"}),
        }
        return w, fmt
    return w, None

def autofit_and_formats(writer, sheet_name, df, fmt):
    if ENGINE == "xlsxwriter":
        ws = writer.sheets[sheet_name]
        ws.freeze_panes(1, 0)
        ws.autofilter(0, 0, len(df), len(df.columns)-1)
        for i, c in enumerate(df.columns):
            if c in {"document_id", "line_item", "cost_center"}:
                ws.set_column(i, i, 20, fmt["text"])
            elif c.lower() in {"value", "impact value", "impact_value", "total_value", "total_impact"}:
                ws.set_column(i, i, 16, fmt["money"])
            elif c.lower() in {"vendor", "vendor_name"}:
                ws.set_column(i, i, 34)
            else:
                ws.set_column(i, i, 18)
    else:
        from openpyxl.utils import get_column_letter
        ws = writer.sheets[sheet_name]
        ws.freeze_panes = "A2"
        ws.auto_filter.ref = ws.dimensions
        for i, _ in enumerate(df.columns, start=1):
            ws.column_dimensions[get_column_letter(i)].width = 18

# ------------------------------
# Date column detection
# ------------------------------
DATE_CANDIDATES = [
    # add your date-like columns here in preferred order
    'purch_doc_date_hpd_po'
]

def choose_date_col(df):
    if DATE_COL and DATE_COL in df.columns:
        return DATE_COL
    for c in DATE_CANDIDATES:
        if c in df.columns:
            try:
                as_dt = pd.to_datetime(df[c], errors="coerce")
                if as_dt.notna().any():
                    return c
            except Exception:
                pass
    raise ValueError(
        "No usable date column found. Set DATE_COL to your column name "
        "or add it to DATE_CANDIDATES."
    )

# ------------------------------
# Period helpers
# ------------------------------
def clamp_to_six_months(df, dcol):
    six_months_ago = RUN_DATE - relativedelta(months=6)
    data_min = pd.to_datetime(df[dcol], errors="coerce").min().date()
    start = max(six_months_ago, data_min)
    return start, RUN_DATE

def iter_daily(start: date, end: date):
    d = start
    while d <= end:
        yield d, d
        d += timedelta(days=1)

def start_of_week_monday(d: date):
    return d - timedelta(days=d.weekday())  # Monday=0

def iter_weekly(start: date, end: date):
    # ISO-style weeks (Mon..Sun); first window clamps to `start`
    w_start = start_of_week_monday(start)
    while w_start <= end:
        w_end = min(w_start + timedelta(days=6), end)
        # clamp the first window's start if range begins mid-week
        s = max(w_start, start)
        yield s, w_end
        w_start = w_start + timedelta(days=7)

def iter_monthly(start: date, end: date):
    m_start = date(start.year, start.month, 1)
    while m_start <= end:
        next_month = (m_start + relativedelta(months=1))
        m_end = min(next_month - timedelta(days=1), end)
        s = max(m_start, start)
        yield s, m_end
        m_start = next_month

# ============================================================
# REPORT 1: Top 10 Vendors & Risky Transactions (your logic)
# ============================================================
def report_top10_vendors_and_txns(df_in, out_path):
    COLUMNS = {
        "vendor_id":   "vendor_or_creditor_acct_no_hpd_po",
        "vendor_name": "vendor_name_1",
        "document_id": "base_id_src_po",
        "risk_level":  "risk_level",
        "cost_center": "plant_src_po",
        "total_value": "on_release_total_value_hpd_po",
        "impact_value":"net_val_po_curr_src_po",
        "main_scenario": "main_risk_scenario",
    }
    FILTER_TO_P2P = True
    RISKY_LEVELS = None  # everything except "No Risk"

    df = df_in.copy()
    present = {k: v for k, v in COLUMNS.items() if v in df.columns}
    df = df.rename(columns={present[k]: k for k in present})

    required = ["vendor_id","vendor_name","document_id","risk_level","total_value"]
    missing = [c for c in required if c not in df.columns]
    if missing:
        return False

    for opt in ["vendor_category","cost_center","impact_value","process","main_scenario","del_flag"]:
        if opt not in df.columns:
            df[opt] = pd.NA

    df = df[~df["del_flag"].fillna("").astype(str).str.upper().eq("L")].copy()

    if FILTER_TO_P2P:
        if "process" in df.columns and df["process"].notna().any():
            df = df[df["process"].fillna("").str.upper().eq("P2P")]
        elif "main_scenario" in df.columns and df["main_scenario"].notna().any():
            df = df[df["main_scenario"].fillna("").str.lower().eq("procurement risk")]

    risk_series = df["risk_level"].fillna("").astype(str).str.strip()
    if RISKY_LEVELS:
        risky_mask = risk_series.str.lower().isin({x.lower() for x in RISKY_LEVELS})
    else:
        risky_mask = (~risk_series.str.lower().eq("no risk")) & (risk_series != "")

    df_risky = df[risky_mask].copy()
    if df_risky.empty:
        return False

    if "impact_value" in df_risky.columns and df_risky["impact_value"].isna().all():
        df_risky["impact_value"] = df_risky["total_value"]
    elif "impact_value" not in df_risky.columns:
        df_risky["impact_value"] = df_risky["total_value"]

    for tcol in ["total_value","impact_value"]:
        df_risky[tcol] = pd.to_numeric(df_risky[tcol], errors="coerce").fillna(0.0)

    for t in ["vendor_id","document_id","cost_center"]:
        if t in df_risky.columns:
            df_risky[t] = df_risky[t].astype("string")

    agg = (
        df_risky
        .groupby(["vendor_id","vendor_name","vendor_category"], dropna=False)
        .agg(
            risky_txn_rows=("document_id","count"),
            distinct_docs=("document_id","nunique"),
            total_risky_value=("total_value","sum"),
            total_impact=("impact_value","sum"),
        ).reset_index()
    )
    top10 = (agg.sort_values(["risky_txn_rows","total_impact"], ascending=[False, False])
                .head(10).reset_index(drop=True))
    if top10.empty:
        return False

    top10_vendor_ids = set(top10["vendor_id"].dropna().astype(str))
    detail_cols = [
        "vendor_id","vendor_name","vendor_category",
        "document_id","risk_level","cost_center",
        "total_value","impact_value"
    ]
    detail_cols = [c for c in detail_cols if c in df_risky.columns]
    detail = (
        df_risky[df_risky["vendor_id"].astype(str).isin(top10_vendor_ids)][detail_cols]
          .sort_values(["vendor_id","impact_value"], ascending=[True, False])
          .reset_index(drop=True)
    )

    writer, fmt = writer_with_formats(out_path)
    top10.to_excel(writer, sheet_name="Top10_Vendors", index=False)
    detail.to_excel(writer, sheet_name="Risky_Txns_Top10", index=False)
    autofit_and_formats(writer, "Top10_Vendors", top10, fmt)
    autofit_and_formats(writer, "Risky_Txns_Top10", detail, fmt)
    writer.close()
    return True

# ============================================================
# REPORT 2: High-Risk Transactions (your logic)
# ============================================================
def report_highrisk_txns(df_in, out_path):
    COLUMNS = {
        "document_id":   'purch_doc_no_src_po',
        "risk_level":    "risk_level",
        "cost_center":   "plant_src_po",
        "vendor_id":     "vendor_or_creditor_acct_no_hpd_po",
        "vendor_name":   "vendor_name_1",
        "line_item":     'purch_doc_item_no_src_po',
        "total_value":   "on_release_total_value_hpd_po",
        "impact_value":  "net_val_po_curr_src_po",
        "main_scenario": "main_risk_scenario",
    }
    FILTER_TO_P2P = True
    RISKY_LEVELS = {"High Risk", "Very High Risk", "Needs Validation"}

    df = df_in.copy()
    present = {k: v for k, v in COLUMNS.items() if v in df.columns}
    df = df.rename(columns={present[k]: k for k in present})

    for opt in ["process","main_scenario","del_flag","impact_value","vendor_id","vendor_name","line_item","value"]:
        if opt not in df.columns:
            df[opt] = pd.NA

    df = df[~df["del_flag"].fillna("").astype(str).str.upper().eq("L")].copy()

    if FILTER_TO_P2P:
        if df["process"].notna().any():
            df = df[df["process"].fillna("").str.upper().eq("P2P")]
        elif df["main_scenario"].notna().any():
            df = df[df["main_scenario"].fillna("").str.lower().eq("procurement risk")]

    risk_series = df["risk_level"].fillna("").astype(str).str.strip()
    df_hr = df[risk_series.str.lower().isin({x.lower() for x in RISKY_LEVELS})].copy()
    if df_hr.empty:
        return False

    for c in ["impact_value","value"]:
        if c in df_hr.columns:
            df_hr[c] = pd.to_numeric(df_hr[c], errors="coerce")
    df_hr["Value"] = df_hr["impact_value"].where(df_hr["impact_value"].notna(), df_hr["value"]).fillna(0.0)

    df_hr["Vendor"] = (
        df_hr.get("vendor_id").astype("string").fillna("")
        .where(df_hr.get("vendor_id").notna(), "")
        .str.replace("<NA>", "", regex=False)
    )
    vn = df_hr.get("vendor_name")
    if vn is not None:
        df_hr["Vendor"] = df_hr["Vendor"].where(df_hr["Vendor"] != "", vn.fillna("").astype(str))
        both = (df_hr["Vendor"] != "") & vn.notna() & (vn.astype(str) != "")
        df_hr.loc[both, "Vendor"] = df_hr.loc[both, "Vendor"] + " – " + vn.loc[both].astype(str)

    for tcol in ["document_id","line_item","cost_center"]:
        if tcol in df_hr.columns:
            df_hr[tcol] = df_hr[tcol].astype("string")

    OUT_COLS = ["document_id", "risk_level", "cost_center", "Vendor", "line_item", "Value"]
    present_out_cols = [c for c in OUT_COLS if c in df_hr.columns]

    top10_txns = (
        df_hr.sort_values("Value", ascending=False)
             .loc[:, present_out_cols]
             .head(10)
             .reset_index(drop=True)
    )
    all_txns = (
        df_hr.loc[:, present_out_cols]
             .sort_values(["risk_level","Value"], ascending=[True, False])
             .reset_index(drop=True)
    )

    if top10_txns.empty and all_txns.empty:
        return False

    writer, fmt = writer_with_formats(out_path)
    top10_txns.to_excel(writer, sheet_name="Top10_HighRisk_Txns", index=False)
    all_txns.to_excel(writer, sheet_name="All_HighRisk_Txns", index=False)
    autofit_and_formats(writer, "Top10_HighRisk_Txns", top10_txns, fmt)
    autofit_and_formats(writer, "All_HighRisk_Txns", all_txns, fmt)
    writer.close()
    return True

# ============================================================
# REPORT 3: Split PO (your logic)
# ============================================================
def report_split_po(df_in, out_path):
    COLUMNS = {
        "document_id":   "purch_doc_no_src_po",
        "risk_level":    "risk_level",
        "cost_center":   "plant_src_po",
        "vendor_id":     "vendor_or_creditor_acct_no_hpd_po",
        "vendor_name":   "vendor_name_1",
        "line_item":     "purch_doc_item_no_src_po",
        "value":         "on_release_total_value_hpd_po",
        "impact_value":  "net_val_po_curr_src_po",
        "main_scenario": "main_risk_scenario",
        "sub_risk_1":    "sub_risk_1",
        "sub_risk_2":    "sub_risk_2",
    }
    FILTER_TO_P2P = True
    split_pattern = re.compile(r"\bsplit[\s_-]*po\b", flags=re.IGNORECASE)

    df = df_in.copy()
    present_map = {k: v for k, v in COLUMNS.items() if v in df.columns}
    df = df.rename(columns={present_map[k]: k for k in present_map})

    for opt in ["process","main_scenario","del_flag","vendor_id","vendor_name","line_item","value","impact_value",
                "sub_risk_1","sub_risk_2"]:
        if opt not in df.columns:
            df[opt] = pd.NA

    df = df[~df["del_flag"].fillna("").astype(str).str.upper().eq("L")].copy()

    if FILTER_TO_P2P:
        if df["process"].notna().any():
            df = df[df["process"].fillna("").str.upper().eq("P2P")]
        elif df["main_scenario"].notna().any():
            df = df[df["main_scenario"].fillna("").str.lower().eq("procurement risk")]

    def is_split_flag(val):
        if pd.isna(val): return False
        if isinstance(val, (list, tuple, set)):
            return any(bool(split_pattern.search(str(x))) for x in val)
        return bool(split_pattern.search(str(val)))

    risk_cols_to_check = [c for c in ["sub_risk_1","sub_risk_2"] if c in df.columns]
    if not risk_cols_to_check:
        return False

    split_mask = False
    for c in risk_cols_to_check:
        split_mask = split_mask | df[c].apply(is_split_flag)

    df_split = df[split_mask].copy()
    if df_split.empty:
        return False

    df_split["Vendor"] = df_split.get("vendor_id").astype("string").fillna("")
    vn = df_split.get("vendor_name")
    if vn is not None:
        df_split["Vendor"] = df_split["Vendor"].mask(df_split["Vendor"].eq(""), vn.fillna("").astype(str))
        both = df_split["Vendor"].ne("") & vn.notna() & (vn.astype(str) != "")
        df_split.loc[both, "Vendor"] = df_split.loc[both, "Vendor"] + " – " + vn.loc[both].astype(str)

    for tcol in ["document_id","line_item","cost_center"]:
        if tcol in df_split.columns:
            df_split[tcol] = df_split[tcol].astype("string")

    if "impact_value" in df_split.columns:
        df_split["impact_value"] = pd.to_numeric(df_split["impact_value"], errors="coerce").fillna(0.0)
    else:
        df_split["impact_value"] = 0.0

    OUT_COLS = ["document_id", "risk_level", "cost_center", "Vendor", "line_item", "impact_value"]
    present_out = [c for c in OUT_COLS if c in df_split.columns]
    split_txns = (df_split.loc[:, present_out]
                        .sort_values("impact_value", ascending=False, kind="mergesort")
                        .reset_index(drop=True))

    writer, fmt = writer_with_formats(out_path)
    split_txns.to_excel(writer, sheet_name="SplitPO_Txns", index=False)
    autofit_and_formats(writer, "SplitPO_Txns", split_txns, fmt)
    writer.close()
    return True

# ============================================================
# REPORT 4: Price Variance (your logic)
# ============================================================
def report_price_variance(df_in, out_path):
    COLUMNS = {
        "document_id":   "purch_doc_no_src_po",
        "risk_level":    "risk_level",
        "cost_center":   "plant_src_po",
        "vendor_id":     "vendor_or_creditor_acct_no_hpd_po",
        "vendor_name":   "vendor_name_1",
        "line_item":     "purch_doc_item_no_src_po",
        "value":         "on_release_total_value_hpd_po",
        "impact_value":  "net_val_po_curr_src_po",
        "main_scenario": "main_risk_scenario",
        "sub_risk_1":    "sub_risk_1",
        "sub_risk_2":    "sub_risk_2",
    }
    FILTER_TO_P2P = True
    pv_pattern = re.compile(r"\bprice[\s_-]*variance\b", flags=re.IGNORECASE)

    df = df_in.copy()
    present_map = {k: v for k, v in COLUMNS.items() if v in df.columns}
    df = df.rename(columns={present_map[k]: k for k in present_map})

    for opt in ["process","main_scenario","del_flag","vendor_id","vendor_name","line_item","value","impact_value",
                "sub_risk_1","sub_risk_2"]:
        if opt not in df.columns:
            df[opt] = pd.NA

    df = df[~df["del_flag"].fillna("").astype(str).str.upper().eq("L")].copy()

    if FILTER_TO_P2P:
        if df["process"].notna().any():
            df = df[df["process"].fillna("").str.upper().eq("P2P")]
        elif df["main_scenario"].notna().any():
            df = df[df["main_scenario"].fillna("").str.lower().eq("procurement risk")]

    def is_pv_flag(val):
        if pd.isna(val): return False
        if isinstance(val, (list, tuple, set)):
            return any(bool(pv_pattern.search(str(x))) for x in val)
        return bool(pv_pattern.search(str(val)))

    risk_cols_to_check = [c for c in ["sub_risk_1","sub_risk_2"] if c in df.columns]
    if not risk_cols_to_check:
        return False

    pv_mask = False
    for c in risk_cols_to_check:
        pv_mask = pv_mask | df[c].apply(is_pv_flag)

    df_pv = df[pv_mask].copy()
    if df_pv.empty:
        return False

    df_pv["Vendor"] = df_pv.get("vendor_id").astype("string").fillna("")
    vn = df_pv.get("vendor_name")
    if vn is not None:
        df_pv["Vendor"] = df_pv["Vendor"].mask(df_pv["Vendor"].eq(""), vn.fillna("").astype(str))
        both = df_pv["Vendor"].ne("") & vn.notna() & (vn.astype(str) != "")
        df_pv.loc[both, "Vendor"] = df_pv.loc[both, "Vendor"] + " – " + vn.loc[both].astype(str)

    for tcol in ["document_id","line_item","cost_center"]:
        if tcol in df_pv.columns:
            df_pv[tcol] = df_pv[tcol].astype("string")

    if "impact_value" in df_pv.columns:
        df_pv["impact_value"] = pd.to_numeric(df_pv["impact_value"], errors="coerce").fillna(0.0)
    else:
        df_pv["impact_value"] = 0.0

    OUT_COLS = ["document_id", "risk_level", "cost_center", "Vendor", "line_item", "impact_value"]
    present_out = [c for c in OUT_COLS if c in df_pv.columns]
    pv_txns = (df_pv.loc[:, present_out]
                    .sort_values("impact_value", ascending=False, kind="mergesort")
                    .reset_index(drop=True))

    writer, fmt = writer_with_formats(out_path)
    pv_txns.to_excel(writer, sheet_name="PriceVariance_Txns", index=False)
    autofit_and_formats(writer, "PriceVariance_Txns", pv_txns, fmt)
    writer.close()
    return True

# ============================================================
# RUNNER (folderized: Daily/Weekly/Monthly subfolders)
# ============================================================
def ensure_base_dirs():
    for sub in ["Daily", "Weekly", "Monthly"]:
        os.makedirs(os.path.join(BASE_OUT, sub), exist_ok=True)

def weekly_folder_name(start_d: date, end_d: date):
    iso_year, iso_week, _ = start_d.isocalendar()
    return f"{iso_year}-W{iso_week:02d}_{start_d:%Y-%m-%d}_to_{end_d:%Y-%m-%d}"

def monthly_folder_name(start_d: date, end_d: date):
    # Use calendar month of start
    return f"{start_d:%Y-%m}"

def run_for_period(df_full, start_d: date, end_d: date, granularity: str):
    # slice by __report_date (precomputed)
    mask = (df_full["__report_date"].dt.date >= start_d) & (df_full["__report_date"].dt.date <= end_d)
    df_slice = df_full.loc[mask].copy()
    if df_slice.empty:
        return

    # choose subfolder per granularity
    if granularity == "Daily":
        subfolder = f"{start_d:%Y-%m-%d}"
    elif granularity == "Weekly":
        subfolder = weekly_folder_name(start_d, end_d)
    elif granularity == "Monthly":
        subfolder = monthly_folder_name(start_d, end_d)
    else:
        subfolder = f"{start_d:%Y-%m-%d}_to_{end_d:%Y-%m-%d}"

    folder = os.path.join(BASE_OUT, granularity, subfolder)
    os.makedirs(folder, exist_ok=True)

    # filenames are short; uniqueness comes from folder structure
    if RUN_TOP10_VENDORS_AND_TXNS:
        path = os.path.join(folder, "Top 10 high-risk vendors by risky transactions.xlsx")
        ok = report_top10_vendors_and_txns(df_slice, path)
        if ok: print(f"[{granularity}] {subfolder} → Top10 Vendors ✔")

    if RUN_HIGHRISK_TXNS:
        path = os.path.join(folder, "Top 10 high-risk transactions.xlsx")
        ok = report_highrisk_txns(df_slice, path)
        if ok: print(f"[{granularity}] {subfolder} → High-Risk Txns ✔")

    if RUN_SPLIT_PO:
        path = os.path.join(folder, "Split PO Report.xlsx")
        ok = report_split_po(df_slice, path)
        if ok: print(f"[{granularity}] {subfolder} → Split PO ✔")

    if RUN_PRICE_VARIANCE:
        path = os.path.join(folder, "Price Variance Risk Report.xlsx")
        ok = report_price_variance(df_slice, path)
        if ok: print(f"[{granularity}] {subfolder} → Price Variance ✔")

def generate_all_reports(po_data: pd.DataFrame):
    if po_data is None or len(po_data) == 0:
        raise ValueError("po_data is empty or not provided.")

    dcol = choose_date_col(po_data)
    df_full = po_data.copy()
    df_full["__report_date"] = pd.to_datetime(df_full[dcol], errors="coerce")
    df_full = df_full[df_full["__report_date"].notna()].copy()
    if df_full.empty:
        raise ValueError(f"No valid dates found in '{dcol}'.")

    start, end = clamp_to_six_months(df_full, "__report_date")
    ensure_base_dirs()

    # Daily
    for s, e in iter_daily(start, end):
        run_for_period(df_full, s, e, "Daily")

    # Weekly (Mon..Sun)
    for s, e in iter_weekly(start, end):
        run_for_period(df_full, s, e, "Weekly")

    # Monthly (calendar)
    for s, e in iter_monthly(start, end):
        run_for_period(df_full, s, e, "Monthly")

# ==========================
# Run:
generate_all_reports(po_data)
# ==========================


[Daily] 2025-04-02 → Top10 Vendors ✔
[Daily] 2025-04-02 → High-Risk Txns ✔
[Daily] 2025-04-02 → Split PO ✔
[Daily] 2025-04-02 → Price Variance ✔
[Daily] 2025-04-03 → Top10 Vendors ✔
[Daily] 2025-04-03 → High-Risk Txns ✔
[Daily] 2025-04-03 → Split PO ✔
[Daily] 2025-04-03 → Price Variance ✔
[Daily] 2025-04-04 → Top10 Vendors ✔
[Daily] 2025-04-04 → High-Risk Txns ✔
[Daily] 2025-04-04 → Split PO ✔
[Daily] 2025-04-06 → Top10 Vendors ✔
[Daily] 2025-04-06 → High-Risk Txns ✔
[Daily] 2025-04-06 → Price Variance ✔
[Daily] 2025-04-07 → Top10 Vendors ✔
[Daily] 2025-04-07 → High-Risk Txns ✔
[Daily] 2025-04-07 → Price Variance ✔
[Daily] 2025-04-08 → Top10 Vendors ✔
[Daily] 2025-04-08 → High-Risk Txns ✔
[Daily] 2025-04-08 → Price Variance ✔
[Daily] 2025-04-09 → Top10 Vendors ✔
[Daily] 2025-04-09 → High-Risk Txns ✔
[Daily] 2025-04-09 → Split PO ✔
[Daily] 2025-04-09 → Price Variance ✔
[Daily] 2025-04-10 → Top10 Vendors ✔
[Daily] 2025-04-10 → High-Risk Txns ✔
[Daily] 2025-04-10 → Split PO ✔
[Daily] 2025